In [1]:
import numpy as np
import pandas as pd

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
df = pd.read_csv('./CBC.csv')

In [3]:
df

,Seq#,ID#,Gender,M,R,F,FirstPurch,ChildBks,YouthBks,CookBks,DoItYBks,RefBks,ArtBks,GeoBks,ItalCook,ItalAtlas,ItalArt,Florence,Related purchase
0,1,25,1,297,14,2,22,0,1,1,0,0,0,0,0,0,0,0,0
1,2,29,0,128,8,2,10,0,0,0,0,0,0,0,0,0,0,0,0
2,3,46,1,138,22,7,56,2,1,2,0,1,0,1,1,0,0,0,2
3,4,47,1,228,2,1,2,0,0,0,0,0,0,0,0,0,0,0,0
4,5,51,1,257,10,1,10,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3996,32950,0,141,2,2,6,0,0,0,0,0,0,0,0,0,0,0,0
3996,3997,32954,1,357,16,9,52,1,1,2,2,2,0,1,0,0,0,0,1
3997,3998,32955,0,48,12,1,12,0,0,0,0,0,0,0,0,0,0,0,0
3998,3999,32976,0,214,14,7,38,1,1,3,1,0,0,0,0,0,0,0,0


* 쓸모없는 변수: Seq#, ID#, Gender, M, R, F, FirstPurch

In [4]:
df.columns[:7]

Index(['Seq#', 'ID#', 'Gender', 'M', 'R', 'F', 'FirstPurch'], dtype='object')

In [5]:
df = df.drop(df.columns[:7], axis = 1)
df

,ChildBks,YouthBks,CookBks,DoItYBks,RefBks,ArtBks,GeoBks,ItalCook,ItalAtlas,ItalArt,Florence,Related purchase
0,0,1,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0
2,2,1,2,0,1,0,1,1,0,0,0,2
3,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0,0,0,0,0,0,0,0,0,0,0,0
3996,1,1,2,2,2,0,1,0,0,0,0,1
3997,0,0,0,0,0,0,0,0,0,0,0,0
3998,1,1,3,1,0,0,0,0,0,0,0,0


#####여러개 구매한 것은 중요하지 않으므로, 1보다 큰 값들은 모두 1로 변환

In [6]:
df.dtypes

ChildBks            int64
YouthBks            int64
CookBks             int64
DoItYBks            int64
RefBks              int64
ArtBks              int64
GeoBks              int64
ItalCook            int64
ItalAtlas           int64
ItalArt             int64
Florence            int64
Related purchase    int64
dtype: object

In [7]:
df[df[df.columns]>1]

,ChildBks,YouthBks,CookBks,DoItYBks,RefBks,ArtBks,GeoBks,ItalCook,ItalAtlas,ItalArt,Florence,Related purchase
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3996,NaN,NaN,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3998,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df[df[df.columns]>1] = 1

In [9]:
df.describe()

,ChildBks,YouthBks,CookBks,DoItYBks,RefBks,ArtBks,GeoBks,ItalCook,ItalAtlas,ItalArt,Florence,Related purchase
count,4000.000000,4000.000000,4000.00000,4000.000000,4000.000000,4000.00000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000
mean,0.394000,0.238250,0.41550,0.254750,0.204750,0.22300,0.266750,0.107500,0.032500,0.043250,0.084500,0.476250
std,0.488696,0.426066,0.49287,0.435775,0.403569,0.41631,0.442316,0.309787,0.177346,0.203445,0.278171,0.499498
min,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,1.00000,1.000000,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.00000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
freq_itemsets = apriori(df, min_support = 400/4000, use_colnames = True)
freq_itemsets

,support,itemsets
0,0.39400,(ChildBks)
1,0.23825,(YouthBks)
2,0.41550,(CookBks)
3,0.25475,(DoItYBks)
4,0.20475,(RefBks)
5,0.22300,(ArtBks)
6,0.26675,(GeoBks)
7,0.10750,(ItalCook)
8,0.47625,(Related purchase)
9,0.14750,"(ChildBks, YouthBks)"


In [11]:
rules = association_rules(freq_itemsets, metric = 'lift', min_threshold = 1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ChildBks),(YouthBks),0.39400,0.23825,0.1475,0.374365,1.571314,0.053629,1.217564
1,(YouthBks),(ChildBks),0.23825,0.39400,0.1475,0.619098,1.571314,0.053629,1.590959
2,(ChildBks),(CookBks),0.39400,0.41550,0.2420,0.614213,1.478251,0.078293,1.515086
3,(CookBks),(ChildBks),0.41550,0.39400,0.2420,0.582431,1.478251,0.078293,1.451256
4,(ChildBks),(DoItYBks),0.39400,0.25475,0.1615,0.409898,1.609022,0.061129,1.262918
...,...,...,...,...,...,...,...,...,...
143,"(Related purchase, CookBks)","(ChildBks, GeoBks)",0.25350,0.14625,0.1095,0.431953,2.953522,0.072426,1.502956
144,(GeoBks),"(ChildBks, Related purchase, CookBks)",0.26675,0.16725,0.1095,0.410497,2.454390,0.064886,1.412630
145,(ChildBks),"(Related purchase, GeoBks, CookBks)",0.39400,0.15625,0.1095,0.277919,1.778680,0.047937,1.168497
146,(Related purchase),"(ChildBks, CookBks, GeoBks)",0.47625,0.10950,0.1095,0.229921,2.099738,0.057351,1.156375


# 'Lift'가 높은 순서대로 정렬

In [12]:
rule = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
rule = rule.sort_values(by = 'lift', ascending = False)
rule

,antecedents,consequents,support,confidence,lift
141,"(ChildBks, Related purchase)","(CookBks, GeoBks)",0.10950,0.463983,2.969492
140,"(CookBks, GeoBks)","(ChildBks, Related purchase)",0.10950,0.700800,2.969492
143,"(Related purchase, CookBks)","(ChildBks, GeoBks)",0.10950,0.431953,2.953522
138,"(ChildBks, GeoBks)","(Related purchase, CookBks)",0.10950,0.748718,2.953522
107,(GeoBks),"(Related purchase, YouthBks)",0.10450,0.391753,2.519309
...,...,...,...,...,...
27,(CookBks),(ArtBks),0.11300,0.271961,1.219558
9,(ChildBks),(ArtBks),0.10525,0.267132,1.197901
8,(ArtBks),(ChildBks),0.10525,0.471973,1.197901
90,(ChildBks),"(ArtBks, Related purchase)",0.10525,0.267132,1.197901


In [13]:
#rule의 인덱스 정렬
rule = rule.reset_index(drop=True)
rule

,antecedents,consequents,support,confidence,lift
0,"(ChildBks, Related purchase)","(CookBks, GeoBks)",0.10950,0.463983,2.969492
1,"(CookBks, GeoBks)","(ChildBks, Related purchase)",0.10950,0.700800,2.969492
2,"(Related purchase, CookBks)","(ChildBks, GeoBks)",0.10950,0.431953,2.953522
3,"(ChildBks, GeoBks)","(Related purchase, CookBks)",0.10950,0.748718,2.953522
4,(GeoBks),"(Related purchase, YouthBks)",0.10450,0.391753,2.519309
...,...,...,...,...,...
143,(CookBks),(ArtBks),0.11300,0.271961,1.219558
144,(ChildBks),(ArtBks),0.10525,0.267132,1.197901
145,(ArtBks),(ChildBks),0.10525,0.471973,1.197901
146,(ChildBks),"(ArtBks, Related purchase)",0.10525,0.267132,1.197901


# rule 데이터프레임에서 조건 및 결과의 길이를 알고싶은 경우?

In [14]:
#len(x)에 대한 설명
var_1 = [1,2,3,4]
print(var_1)

[1, 2, 3, 4]


In [15]:
#리스트의 갯수를 출력할 수 있음
len(var_1)

4

In [16]:
#길이를 출력해주는 함수 생성
#len(x): x의 길이를 출력하는 것
def return_length(x):
  return len(x)

In [17]:
rule['antecedents_length'] = rule['antecedents'].apply(return_length)

In [18]:
rule

,antecedents,consequents,support,confidence,lift,antecedents_length
0,"(ChildBks, Related purchase)","(CookBks, GeoBks)",0.10950,0.463983,2.969492,2
1,"(CookBks, GeoBks)","(ChildBks, Related purchase)",0.10950,0.700800,2.969492,2
2,"(Related purchase, CookBks)","(ChildBks, GeoBks)",0.10950,0.431953,2.953522,2
3,"(ChildBks, GeoBks)","(Related purchase, CookBks)",0.10950,0.748718,2.953522,2
4,(GeoBks),"(Related purchase, YouthBks)",0.10450,0.391753,2.519309,1
...,...,...,...,...,...,...
143,(CookBks),(ArtBks),0.11300,0.271961,1.219558,1
144,(ChildBks),(ArtBks),0.10525,0.267132,1.197901,1
145,(ArtBks),(ChildBks),0.10525,0.471973,1.197901,1
146,(ChildBks),"(ArtBks, Related purchase)",0.10525,0.267132,1.197901,1


## 마찬가지로, 결과절의 길이를 알고싶은 경우?

In [19]:
rule['consequents_length'] = rule['consequents'].apply(return_length)

In [20]:
rule

,antecedents,consequents,support,confidence,lift,antecedents_length,consequents_length
0,"(ChildBks, Related purchase)","(CookBks, GeoBks)",0.10950,0.463983,2.969492,2,2
1,"(CookBks, GeoBks)","(ChildBks, Related purchase)",0.10950,0.700800,2.969492,2,2
2,"(Related purchase, CookBks)","(ChildBks, GeoBks)",0.10950,0.431953,2.953522,2,2
3,"(ChildBks, GeoBks)","(Related purchase, CookBks)",0.10950,0.748718,2.953522,2,2
4,(GeoBks),"(Related purchase, YouthBks)",0.10450,0.391753,2.519309,1,2
...,...,...,...,...,...,...,...
143,(CookBks),(ArtBks),0.11300,0.271961,1.219558,1,1
144,(ChildBks),(ArtBks),0.10525,0.267132,1.197901,1,1
145,(ArtBks),(ChildBks),0.10525,0.471973,1.197901,1,1
146,(ChildBks),"(ArtBks, Related purchase)",0.10525,0.267132,1.197901,1,2


#도출한 규칙의 결과를 바탕으로 해석

In [21]:
rule.loc[rule['antecedents'] == ({'ChildBks', 'GeoBks'}), :]

,antecedents,consequents,support,confidence,lift,antecedents_length,consequents_length
3,"(ChildBks, GeoBks)","(Related purchase, CookBks)",0.10950,0.748718,2.953522,2,2
19,"(ChildBks, GeoBks)",(Related purchase),0.14625,1.000000,2.099738,2,1
68,"(ChildBks, GeoBks)",(CookBks),0.10950,0.748718,1.801969,2,1


## loc

* 딕셔너리 구조로 사용하는 이유?
  * rule['antecedents']의 형태가 ({'a', 'b'})와 같기 때문임

In [22]:
#X.unique(): X의 종류를 출력하는 것
#rule['antecedents']의 0번째 종류 출력

rule['antecedents'].unique()[0]

frozenset({'ChildBks', 'Related purchase'})

* (참고) frozenset: set(튜플 자료구조)와 동일하지만, 값을 추가할 수 없는 형태

  * 따라서 loc의 조건이 위와 같은 자료형태로 입력되어야 함

In [23]:
rule.loc[rule['antecedents'] == ({'ChildBks', 'GeoBks'}), :]

,antecedents,consequents,support,confidence,lift,antecedents_length,consequents_length
3,"(ChildBks, GeoBks)","(Related purchase, CookBks)",0.10950,0.748718,2.953522,2,2
19,"(ChildBks, GeoBks)",(Related purchase),0.14625,1.000000,2.099738,2,1
68,"(ChildBks, GeoBks)",(CookBks),0.10950,0.748718,1.801969,2,1


## 이는 조건절이 'ChildBks', 'GeoBks'인 경우에

* Support, Confidence, Lift를 알 수 있음

>찾고싶은 여러 규칙들을 조건으로 입력하여, 연관규칙이 있는지 확인할 수 있음